# Workshop 3: Exploring IllustrisTNG simulations to derive observationally comparable star formation rates and metallicities

## Notebook 1: IllustrisTNG API and downloading data

Created by Bryanne McDonough and Olivia Curtis
In this notebook, you will be introduced to the different data types availbale for the TNG simulations, and how to use the TNG API to download the data you need. If you haven't download iapi_TNG.py here: https://github.com/bryannemcd/TNG_workshop/blob/main/iapi_TNG.py

IMPORTANT: You will need to update iapi_TNG.py on your machine with the API key you requested from the TNG website. If you do not have an API key, request one here: https:/www.tng-project.org/users/register/

Below, you will find several exercises to get practice working with the different data types available. There are also 'extensions,' prompts for you to explore data that is aligned with your interests.

In [ ]:
import iapi_TNG as iapi
#this package contains useful functions for downloading the neccessary data
#make sure you have edited iapi_TNG.py to include your personal API key
import numpy as np
import h5py #most TNG data is downloaded as hdf5 files
import matplotlib.pyplot as plt
import os.path

 
baseUrl = 'http://www.tng-project.org/api/'


## General simulation data

Using the TNG API, we can explore basic information about the simulations available.

First, define your working directory.

In [ ]:
####EDIT THIS FOR YOUR MACHINE#####
dirc='/projectnb/res-star/TNG_workshop/'

Then, define which simulation you would like to look at. The options include:

1. TNG100-1 is the highest resolution simulation in the 100 Mpc simulation box
2. TNG50-1 is the highest resolution available in a 50 Mpc box
3. TNG300-1 is the largest volume simulation (300 Mpc)

Here, Lower resolutions are available with -N replacing -1, allowing for testing resolution dependency.
'Dark' simulations are also available that are are dark-matter only runs. Lastly, subboxes are availble that provide higher time resolution

For the exercises in this worrkshop, you will need to use a baryonic simulation, we recommend TNG100-1, TNG50-1, or TNG300-1.

In [ ]:
###specify which simulation you want to explore###
sim='TNG100-1'

You can check all available simulations by running the following code block.

In [ ]:
r=iapi.get(baseUrl)
print(r)

Now, let's investigate the properties of the simulation that you have selected.

To get the URL of the simulation that you are using, append the variable sim to the variable baseUrl that we defined above.

In [ ]:
#check the properties of the simulation you have selected
simUrl = baseUrl+sim
print(simUrl) 

Feel free to follow that URL to view it in your browser (make sure that you are logged in first!)

Next, we can retrieve and print the metadeta of the specific simulation using iapi.get().

In [ ]:
simdata = iapi.get(simUrl)
print(simdata['description'])

That is, simdata contains all of the simulation-level metadata for the simulation, saved as a Python dictionary. You can view all of the keys to the dictionary by running the following code block.

In [ ]:
simdata.keys()

### Exercise (don't skip!): Find value of Hubble's constant used in the simulation you chose to explore
In simulations, units often include 'little h' or Hubble's constant divided by 100.

In these simulations, the value of h is stored in the simulation data as the key 'hubble'.

In the following code block, determine the value of h from the simulation's metadata. (Hint: should find h=0.6774)

In [ ]:
h = #Fill in this code

### Group catalogs

Group catalogs contain properies of all identified halos or subhalos (galaxies) in a given snapshot. These are good for obtaining masses, positions, and other global properties. You can check out details about the available fields here: https://www.tng-project.org/data/docs/specifications/#sec2 

In iapi_TNG there are two similar functions that obtain a field for all subhalos or all halos in a given simulation at a given snapshot:

> getSubhaloField(field, simulation='TNG100-1', snapshot=99, fileName='tempCat', rewriteFile=1)

> getHaloField(field, simulation='TNG100-1', snapshot=99, fileName='tempCat', rewriteFile=1)

- field (str): name of field to be returned from the table linked above, e.g. 'SubhaloPos'
- simulation (str): name of simulation, e.g. 'TNG100-1'
- snapshot (int): snapshot to pull data from. For TNG, snapshot=99 is z=0, which is the default
- fileName (str): path to the file where you want to save the data, recommended to avoid repeated API requests
- rewriteFile (0 or 1): if 0 (recommended), will attempt to pull from an existing file (fileName) before downloading; if 1 will download and overwrite

Now let's fetch the fields that we will want for our later analysis.

First, the 'SubhaloFlag' indicates whether a subhalo is cosmological in origin. Generally, we only want to use data from subhalos that have flag=1.

For now, we will pull the subhalo flags with

In [ ]:
flag=iapi.getSubhaloField('SubhaloFlag',simulation=sim,fileName=dirc+'catalogs/SubhaloFlag',rewriteFile=0)

Next, we can pull all of the positions of the subhalos. Note that the positions are in units of comoving Kpc/h.

In [ ]:
pos=iapi.getSubhaloField('SubhaloPos',fileName='catalogs/SubhaloPos',rewriteFile=0)

Next, let's fetch a field that will tell us about the mass of the galaxy. 'SubhaloMassType' gives the total mass of all bound particles, separated by particle type.

In [ ]:
#let's fetch a field that will tell us about the mass of the galaxy
#SubhaloMassType gives the total mass of all bound particles, separated by particle type
mass=iapi.getSubhaloField('SubhaloMassType',simulation=sim,fileName=dirc+'catalogs/MassType',rewriteFile=0)
print(mass.shape)

Notice that there are 6 entries for each subhalo, one for each particle type:

0. gas
1. dark matter
2. unused
3. tracers (you can ignore these)
4. stars/wind
5. black holes.

To pull the stellar masses, we can splice the previous with the following code.

In [ ]:
#Pull the stellar mass: 
stellar_mass=mass[:,4]

According to the documentation, these masses are given in units of 10^10 M_\odot / h. To convert those to physical units we can run.

In [ ]:
stellar_mass=stellar_mass*10**10/h
#running into an error? did you define h above?

We will also be working with star formation rates. 

The subhalo catalog includes 'SubhaloSFR', which is the sum of SFRs over all gas particles bound to the subhalo (i.e., an instantaneous star formation rate).
These are NOT directly comparable to SFRs obtained from observations since observational tracers generally detect already formed stars, not stars about to be formed

If we want to get more comparable SFRs, we'll have to dig into particle data or merger trees, which we will get to later.

For now, let's pull the instantaneous star formation rates.

In [ ]:
sfr_inst = iapi.getSubhaloField('SubhaloSFR',simulation = sim,fileName=dirc+'SubhaloSFR',rewriteFile=0)

### Exercise:

There are several other fields relating to galaxy mass. Review those found at the link above and fetch at least one other field relating to mass. Later, we will test the effect of using other definitions of mass on the global star formation main sequence. Generally, you will want to use a mass that is most comparable to how mass was measured in observations you want to compare to.


In [ ]:
### Pull an additional stellar mass field ###



In [ ]:
### Convert this mass field to physical units ###



### Exercise: Metallicities
Interested in metallicities? Reveiw the fields availalbe in the data specifications. Make sure to scroll all the way through, there are many different metallicity fields, based on both stars and gas. 

In [ ]:
### Download metallicity fields ###


### Data Preprocessing

We need to clean our data before we can use them. This usually involves making cuts to galaxy masses, magnitudes, star formation rates, etc.

First, it is useful to keep track of each subhalo's ID (i.e., subID), which is the objects index into the fields.

In [ ]:
subID=np.arange(0,len(stellar_mass))

Next, let's mask out any galaxy that is not cosmological in origin and that has stellar masses <= 10^8 solar masses.

In [ ]:
mask = (flag==1) & (stellar_mass>10**8)

print("Before trimming, there are ", len(stellar_mass), ' subhalos in our catalog')

IDs=subID[mask]
s_mass=stellar_mass[mask]
sfr_i = sfr_inst[mask]

print("After trimming, there are ", len(s_mass), ' subhalos in our catalog')

Lastly, we will save these fields as a Python dictionary called galcat. We will be using this catalog throughout the rest of this workshop.

In [ ]:
galcat = {
    'subID' : IDs,
    'pos' : pos,
    'M_*' : s_mass,
    'SFR_inst': sfr_i
}

#save the galaxy catalog for later use
np.save(dirc+'galcat', galcat)

## Merger Trees

Tracing a subhalo through cosmic time can be complicated by the major and minor mergers that ultimately form a z=0 galaxy. The merger trees trace the most massive progenitor of a subhalos through previous snapshots. See the TNG data specifications for more information: https://www.tng-project.org/data/docs/specifications/#sec2

In this workshop, we will be using the SubLink merger trees.

iapi_TNG contains the function gettree(snapnum,subID), which obtains the tree for a given galaxy. The trees contain all the fields in the Halo and Subhalo group catalogs, for each snapshot. Subhalo information will always be for the progenitor of the subID at snapnum. The group/halo of a subhalo may change, so the group information in previous snapshots may not be for the group the subhalo is a member of at snapnum.

getredshift(snapnum) is another useful function in iapi_TNG. This returns the redfshift of a given snapshot. 

For this notebook, let's work with a single subhalo at a subhalo in the top middle of our sample (where galaxy indices are very roughly ordered from largest to smalles). Specifically, let's work with the galaxy whose index into IDs is 24957 (this object has a corresponding ID of 565261).

In [ ]:
this_index = 24957
sub=IDs[this_index]
print(sub)

Next, let's use iapi.gettree() to pull the merger tree of this subhalo.

In [ ]:
subTreeFile = iapi.gettree(99,sub)
print(subTreeFile)

This just saved the file '/trees/sublink_mpb_565261.hdf5' into your working directory. We can open that .hdf5 file using h5py.

In [ ]:
#open the hdf5 file that contains the tree
subTree = h5py.File(subTreeFile,'r')

We can then check the fields that we have to work with.

In [ ]:
print(subTree.keys())

Next, we will pull the snapshot numbers that correspond to each entry in each of the fields for this tree.

In [ ]:
#pull the snapshot numbers that correspond to each entry in each of the fields for this tree
snaps = subTree['SnapNum'][:]
print(snaps)

Notice how the first entry corresponds to z=0! The latest entries are first, while the earliest entries are last

Some subhalos have shorter merger trees than others. This particular merger tree indicates that the pregenitor of our test subhalo formed during snapshot 5. 

We can determine what redshift this corresponds to using iapi.getredshift()

In [ ]:
#What redshift does the earliest snapshot correspond to?
formation_redshift = iapi.getredshift(snaps[-1])

print("The progenitor of our test subhalo formed during redshift z = ", formation_redshift)

We can also do this generally for all snapshots.

In [ ]:
#construct an array of redshifts corresponding to snaps
z = iapi.getredshift(snaps)
print(z)

### Exercise: Construct a plot of star formation rate versus redshift for this subhalo


In [ ]:
### Plot SFR versus redshift for this subhalo ###
#hint: pull the SubhaloSFR field from the merger tree first
import matplotlib.pyplot as plt



### Extension: Plot other properties versus redshift for this subhalo
What other properties would be interested to explore as a function of redshift?

In [ ]:
### Extension ###

### Particle Data
Now, we can download some particle data that will allow us to compute luminosity-weighted ages and time-averaged star formation rates for a galaxy. Full snapshots contain way more data than we need, so we can instead pull the parameters we want from 'cutouts' - files that contain data for all particles bound to a subhalo.

First, define the particle type that we are interested in

In [ ]:
parttype='stars'

as well as the particle fields that we want to pull.

In [ ]:
particle_fields = 'Coordinates,Masses,GFM_Metallicity,GFM_StellarFormationTime,GFM_InitialMass,GFM_StellarPhotometrics'

Note that for time-averaged SFR calculations, the initial mass of a star should be used.

Lastly, use iapi.getSubcutout to pull the particle data for each of these fields.

In [ ]:
cut_file = iapi.getSubcutout(sub, \
                             parttype, \
                             particle_fields, \
                             sim=sim, \
                             snapnum=99, \
                             fName=dirc+'cutouts/'+parttype+'_'+str(sub))
print(cut_file)

#running into issues? follow the sub_url that getSubcutout prints to check if a cutout exists for this subhalo

This cutout file was saved to the file /cutouts/stars_565261.hdf5 in your work directory. Feel free to open this catalog on your own and explore the particle data yourself.

### Supplementary Data Catalogs
Rather than instantaneous star formation rates from gas particles, a more observationally-comparable measure is time-averaged star formation rates. Later in this workshop we will cover how to obtain these yourself, but for now we can make use of a supplementary catalog: https://www.tng-project.org/data/docs/specifications/#sec5b

First, we can download the catalog for the simulation of our choice, but note that some catalogs are only available for certain simulations.

In [ ]:
#this is a large file, it may take awhile to download
SFRf=dirc+'SubhaloSFR_ta'

#If statement checks to see if the file already exists
#it's here to make sure you do not download
#this large file again by accident!
if os.path.exists(SFRf): 
    sfr_cat=SFRf+'.hdf5'
else: 
    sfr_cat = iapi.get('https://www.tng-project.org/api/'+sim+'/files/star_formation_rates.hdf5', fName=SFRf)

Then, we can open the supplementary catalog that corresponds to the snapshot of interest with h5py.

In [ ]:
supp_catalog = h5py.File(sfr_cat,'r')

Looking at its keys, we see it has an entry for multiple snapshots.

In [ ]:
supp_catalog.keys()

Let's load the dictionary that corresponds to snapshot 99 and look at its keys.

Notice that there are several options. There are SFRs that are calculated within certain apertures and SFRs calculated over different timescales. When comparing simulations to observations, it's important to use a timescale comparable to the observational tracer (e.g. Halpha roughly traces 10Myrs)

In [ ]:
supp_catalog_snap99 = supp_catalog['Snapshot_99']
print(supp_catalog_snap99.keys())

Let's pull the fields 'SubfindID' and 'SFR_MsunPerYrs_in_all_10Myrs' which pull their subfindID and star formation rate in the past 10 Myrs, respectively. 

In [ ]:
SFR_subID = supp_catalog_snap99['SubfindID'][:]
SFR_all_10 = supp_catalog_snap99['SFR_MsunPerYrs_in_all_10Myrs'][:]

Recall that we had previously trimmed the galaxy catalog that we were using. If we want to append these time-averaged star formation rates to that catalog, then we will need to match the subIDs between the two catalogs.

First, make an empty array that is the same shape as galcat. We will populate this array with time-averaged star formation rates.

In [ ]:
SFR_10=np.empty(len(IDs))

Next, let's loop over every entry in galcat. We will then find the entry in SFR_subID (from the supplementary catalog) that has the same subID as the current galcat entry.

In [ ]:
#For each entry in galcat:
for i in range(0,len(IDs)):
    
    #we only want to match our
    #current entry in galcat with its
    #corresponding entry in the 
    #supplementary catalog.
    mask = SFR_subID==IDs[i]
    
    #Try to assign the time-averaged SFR 
    #not all galaxies in our sample may 
    #have SFRs computed in this catalog
    try: 
        SFR_10[i]=SFR_all_10[mask]
    except: 
        SFR_10[i]=np.nan
        
print(SFR_10)
print(SFR_10.shape)

Lastly, let's update our galcat.

In [ ]:
galcat['SFR_all_10']=SFR_10
np.save(dirc+'galcat', galcat)

### Extension: Edit the code above to use a different aperature, or save additional timescales

In [ ]:
### Extension ###

